# Data and Training

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os

# Define the custom dataset class
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.classes = ['car', 'not_car']
        self.data = []
        for idx, class_name in enumerate(self.classes):
            class_dir = os.path.join(root_dir, class_name)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                self.data.append((img_path, idx))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

# Define the transformations
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Create the dataset and dataloader
train_dataset = CustomDataset(root_dir='./Data/train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Define the model architecture
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.fc1 = nn.Linear(64 * 13 * 13, 256)  # Adjust input size based on expected size
        self.fc2 = nn.Linear(256, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 13 * 13)  # Adjust input size based on expected size
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create an instance of the model
model = MyModel()

# Define device
device = torch.device("cpu")

# Move the model to the device
model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Define the training function
def train_model(model, train_loader, criterion, optimizer, num_epochs=40):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 50 == 49:
                print(f"[Epoch {epoch + 1}, Iteration {i + 1}] loss: {running_loss / 50:.3f}")
                running_loss = 0.0

        print(f"Epoch {epoch + 1} average loss: {running_loss / len(train_loader):.3f}")

# Train the model
train_model(model, train_loader, criterion, optimizer, num_epochs=40)


[Epoch 1, Iteration 50] loss: 0.425
[Epoch 1, Iteration 100] loss: 0.145
Epoch 1 average loss: 0.025
[Epoch 2, Iteration 50] loss: 0.077
[Epoch 2, Iteration 100] loss: 0.096
Epoch 2 average loss: 0.015
[Epoch 3, Iteration 50] loss: 0.057
[Epoch 3, Iteration 100] loss: 0.054
Epoch 3 average loss: 0.013
[Epoch 4, Iteration 50] loss: 0.059
[Epoch 4, Iteration 100] loss: 0.052
Epoch 4 average loss: 0.013
[Epoch 5, Iteration 50] loss: 0.052
[Epoch 5, Iteration 100] loss: 0.060
Epoch 5 average loss: 0.013
[Epoch 6, Iteration 50] loss: 0.035
[Epoch 6, Iteration 100] loss: 0.031
Epoch 6 average loss: 0.008
[Epoch 7, Iteration 50] loss: 0.044
[Epoch 7, Iteration 100] loss: 0.038
Epoch 7 average loss: 0.008
[Epoch 8, Iteration 50] loss: 0.021
[Epoch 8, Iteration 100] loss: 0.025
Epoch 8 average loss: 0.009
[Epoch 9, Iteration 50] loss: 0.029
[Epoch 9, Iteration 100] loss: 0.030
Epoch 9 average loss: 0.006
[Epoch 10, Iteration 50] loss: 0.022
[Epoch 10, Iteration 100] loss: 0.025
Epoch 10 average

# Training the Predefined Architecture: MobileNet

In [17]:
import torchvision.models as models

# Load the MobileNet model
mobilenet = models.mobilenet_v2(pretrained=False)

# Modify the last layer to match our number of classes (2)
mobilenet.classifier[1] = nn.Linear(mobilenet.classifier[1].in_features, 2)

# Move the model to the device
mobilenet.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mobilenet.parameters(), lr=0.0001)

# Define the training function for the predefined model
def train_predefined_model(model, train_loader, criterion, optimizer, num_epochs=30):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 50 == 49:
                print(f"[Epoch {epoch + 1}, Iteration {i + 1}] loss: {running_loss / 50:.3f}")
                running_loss = 0.0

        print(f"Epoch {epoch + 1} average loss: {running_loss / len(train_loader):.3f}")

# Train the MobileNet model
train_predefined_model(mobilenet, train_loader, criterion, optimizer, num_epochs=30)


C:\Users\aslam\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\aslam\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


[Epoch 1, Iteration 50] loss: 0.504
[Epoch 1, Iteration 100] loss: 0.416
Epoch 1 average loss: 0.093
[Epoch 2, Iteration 50] loss: 0.319
[Epoch 2, Iteration 100] loss: 0.328
Epoch 2 average loss: 0.062
[Epoch 3, Iteration 50] loss: 0.245
[Epoch 3, Iteration 100] loss: 0.239
Epoch 3 average loss: 0.056
[Epoch 4, Iteration 50] loss: 0.204
[Epoch 4, Iteration 100] loss: 0.185
Epoch 4 average loss: 0.042
[Epoch 5, Iteration 50] loss: 0.171
[Epoch 5, Iteration 100] loss: 0.159
Epoch 5 average loss: 0.038
[Epoch 6, Iteration 50] loss: 0.114
[Epoch 6, Iteration 100] loss: 0.123
Epoch 6 average loss: 0.032
[Epoch 7, Iteration 50] loss: 0.091
[Epoch 7, Iteration 100] loss: 0.106
Epoch 7 average loss: 0.021
[Epoch 8, Iteration 50] loss: 0.061
[Epoch 8, Iteration 100] loss: 0.080
Epoch 8 average loss: 0.020
[Epoch 9, Iteration 50] loss: 0.099
[Epoch 9, Iteration 100] loss: 0.072
Epoch 9 average loss: 0.013
[Epoch 10, Iteration 50] loss: 0.053
[Epoch 10, Iteration 100] loss: 0.063
Epoch 10 average

# Evaluation

In [18]:
# Function to evaluate the model
def evaluate_model(model, test_loader):
    model.eval()
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    cm = confusion_matrix(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    accuracy = accuracy_score(all_labels, all_preds)

    print(f"Confusion Matrix:\n{cm}")
    print(f"Precision: {precision:.3f}")
    print(f"Recall: {recall:.3f}")
    print(f"F1 Score: {f1:.3f}")
    print(f"Accuracy: {accuracy:.3f}")

# Evaluate the custom model
print("Evaluating custom model...")
evaluate_model(model, test_loader)

# Evaluate the MobileNet model
print("Evaluating MobileNet model...")
evaluate_model(mobilenet, test_loader)


Evaluating custom model...
Confusion Matrix:
[[369  31]
 [ 23 923]]
Precision: 0.968
Recall: 0.976
F1 Score: 0.972
Accuracy: 0.960
Evaluating MobileNet model...
Confusion Matrix:
[[336  64]
 [ 38 908]]
Precision: 0.934
Recall: 0.960
F1 Score: 0.947
Accuracy: 0.924
